In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
 
# Training settings
batch_size = 14
 
# MNIST Dataset
train_dataset = datasets.MNIST(root='/pleasant/trainingset.xlsx',
                train=True,
                transform=transforms.ToTensor(),
                download=True)
 
test_dataset1 = datasets.MNIST(root='/pleasant/newodorset.xlsx',
               train=False,
               transform=transforms.ToTensor())

test_dataset2 = datasets.MNIST(root='/pleasant/oilsset.xlsx',
               train=False,
               transform=transforms.ToTensor())
    
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                      batch_size=batch_size,
                      shuffle=False)
 
test_loader1 = torch.utils.data.DataLoader(dataset=test_dataset1,
                     batch_size=12,
                     shuffle=False)
test_loader1 = torch.utils.data.DataLoader(dataset=test_dataset2,
                     batch_size=5,
                     shuffle=False)

 
 
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    # 输入1通道，输出10通道，kernel 5*5
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.mp = nn.MaxPool2d(2)
    # fully connect
    self.fc = nn.Linear(320, 10)
 
  def forward(self, x):
    # in_size = 238
    in_size = x.size(0) # one batch
    # x: 238*10
    x = F.relu(self.mp(self.conv1(x)))
    # x: 238*20
    x = F.relu(self.mp(self.conv2(x)))
    # x: 238*
    x = x.view(in_size, -1) # flatten the tensor
    # x: 238*10
    x = self.fc(x)
    return F.log_softmax(x)
 
 
model = Net()
 
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
 
def train(epoch):
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % 200 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.data[0]))
 
 
def test():
  test_loss = 0
  correct = 0
  for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    # sum up batch loss
    test_loss += F.nll_loss(output, target, size_average=False).data[0]
    # get the index of the max log-probability
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()
 
  test_loss /= len(test_loader.dataset)
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))
 
 
for epoch in range(1, 10):
  train(epoch)
  test()
